In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%%writefile tmp.bpf.c

struct bar_t {
    char c;
} __attribute__((preserve_access_index));

struct foo_t {
  int i;
  int j;
  int k;
  struct bar_t bar;
} __attribute__((preserve_access_index));

__attribute__((section("kprobe/f")))
int f(struct foo_t* foo) { 
    return foo->bar.c;
}

__attribute__((section("kprobe/g")))
int g(struct foo_t* foo) { 
    int i = foo->i;
    int k = foo->k;
    return i + k;
}

Overwriting tmp.bpf.c


In [3]:
!clang -g -O2 -target bpf -c tmp.bpf.c -o tmp.bpf.o
!llvm-objdump-18 --disassemble --reloc --source tmp.bpf.o


tmp.bpf.o:	file format elf64-bpf

Disassembly of section kprobe/f:

0000000000000000 <f>:
;     return foo->bar.c;
       0:	71 10 0c 00 00 00 00 00	r0 = *(u8 *)(r1 + 0xc)
		0000000000000000:  CO-RE <byte_off> [2] struct foo_t::bar.c (0:3:0)
       1:	67 00 00 00 38 00 00 00	r0 <<= 0x38
       2:	c7 00 00 00 38 00 00 00	r0 s>>= 0x38
       3:	95 00 00 00 00 00 00 00	exit

Disassembly of section kprobe/g:

0000000000000000 <g>:
;     int i = foo->i;
       0:	61 12 00 00 00 00 00 00	r2 = *(u32 *)(r1 + 0x0)
		0000000000000000:  CO-RE <byte_off> [2] struct foo_t::i (0:0)
;     int k = foo->k;
       1:	61 10 08 00 00 00 00 00	r0 = *(u32 *)(r1 + 0x8)
		0000000000000008:  CO-RE <byte_off> [2] struct foo_t::k (0:2)
;     return i + k;
       2:	0f 20 00 00 00 00 00 00	r0 += r2
       3:	95 00 00 00 00 00 00 00	exit


In [4]:
# !hexdump -C /tmp/struct.o

In [15]:
from depsurf import BPFObjectFile

bpfobj = BPFObjectFile("tmp.bpf.o")

# bpfobj.get_section_info()

bpfobj.get_relo_info()

[system.py:19] INFO: Running command: "/usr/lib/linux-tools/6.2.0-39-generic/bpftool btf dump file tmp.bpf.o format c > tmp.bpf.h"
[system.py:19] INFO: Running command: "/usr/lib/linux-tools/6.2.0-39-generic/bpftool btf dump file tmp.bpf.o format raw > tmp.bpf.txt"
[system.py:19] INFO: Running command: "/usr/lib/linux-tools/6.2.0-39-generic/bpftool btf dump file tmp.bpf.o --json > tmp.bpf.json"


[BTFReloSection(sec_name='kprobe/f',
                relocations=[BTFReloEntry(insn_off=0,
                                          types=[('STRUCT', 'foo_t', 'bar'),
                                                 ('STRUCT', 'bar_t', 'c')],
                                          access_str='0:3:0',
                                          kind=<BTFCoreReloKind.FIELD_BYTE_OFFSET: 0>)]),
 BTFReloSection(sec_name='kprobe/g',
                relocations=[BTFReloEntry(insn_off=0,
                                          types=[('STRUCT', 'foo_t', 'i')],
                                          access_str='0:0',
                                          kind=<BTFCoreReloKind.FIELD_BYTE_OFFSET: 0>),
                             BTFReloEntry(insn_off=8,
                                          types=[('STRUCT', 'foo_t', 'k')],
                                          access_str='0:2',
                                          kind=<BTFCoreReloKind.FIELD_BYTE_OFFSET: 0>)])]